In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [18]:
# %tensorflow_version 2.x
# import tensorflow as tf
# import timeit

# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   print(
#       '\n\nThis error most likely means that this notebook is not '
#       'configured to use a GPU.  Change this in Notebook Settings via the '
#       'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
#   raise SystemError('GPU device not found')

# def cpu():
#   with tf.device('/cpu:0'):
#     random_image_cpu = tf.random.normal((100, 100, 100, 3))
#     net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
#     return tf.math.reduce_sum(net_cpu)

# def gpu():
#   with tf.device('/device:GPU:0'):
#     random_image_gpu = tf.random.normal((100, 100, 100, 3))
#     net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
#     return tf.math.reduce_sum(net_gpu)
  
# # We run each op once to warm up; see: https://stackoverflow.com/a/45067900
# cpu()
# gpu()

# # Run the op several times.
# print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
#       '(batch x height x width x channel). Sum of ten runs.')
# print('CPU (s):')
# cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
# print(cpu_time)
# print('GPU (s):')
# gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
# print(gpu_time)
# print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.13971086499987
GPU (s):
0.1043293469992932
GPU speedup over CPU: 30x


In [2]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [3]:
batch_size = 64
image_size = 64 #64x64 is the image size

In [4]:
transform = transforms.Compose([transforms.Resize(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
dataset = dset.CIFAR10(root = './data', download = True, transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle = True, num_workers=2)

Files already downloaded and verified


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
os.chdir('/content/drive/My Drive/GAN')
os.getcwd()

'/content/drive/My Drive/GAN'

In [8]:
def weights_init(m):    #m is a neural network 
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)

    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)    

In [9]:
class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), #100 dims for random noise ip (channel dims)
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        ) 

    def forward(self, input):
        output = self.main(input)
        return output

netG = G()
netG.apply(weights_init)


G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [10]:
class D(nn.Module):

      def __init__(self):
          super(D, self).__init__()
          self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()   #Returns a matrix of 0-1 discriminatoy valus where 0 is reject and 1 is accept
          )


      def forward(self, input):
          output = self.main(input)
          return output.view(-1)


netD = D()
netD.apply(weights_init)          

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [11]:
criterion = nn.BCELoss()
optim_D = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optim_G = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))

In [ ]:
for epoch in range(17):
    for i, data in enumerate(dataloader, 0):   #This holds a batch at a time, and we can perform minibatch SGD, index 0 is to denote the start value of i

        #Discriminator training
        netD.zero_grad()
        real, _ = data #return value 2 will hold class info
        input = Variable(real)
        output = netD(input)
        target = Variable(torch.ones(input.size()[0]))
        errD_real = criterion(output, target)

        #Fake data
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) #Variable with noise, 100 matrices of 1x1 each (acc to network size req)
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())  #fake has grad info from G n/w, it is not needed , speed up the computation by detaching
        errD_fake = criterion(output, target)

        d_error = errD_real + errD_fake
        d_error.backward()
        optim_D.step()


        #Generator training
        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake) #do not detach the gradient as we need it for backprop in G
        g_error = criterion(output, target)
        g_error.backward()
        optim_G.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), d_error.data, g_error.data))

        if i % 100 == 0:

            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' %("./results", epoch), normalize = True)



[0/25][0/782] Loss_D: 0.5330 Loss_G: 6.0857
[0/25][1/782] Loss_D: 0.4194 Loss_G: 4.2773
[0/25][2/782] Loss_D: 0.4782 Loss_G: 8.1539
[0/25][3/782] Loss_D: 1.1990 Loss_G: 1.3737
[0/25][4/782] Loss_D: 2.0992 Loss_G: 11.3177
[0/25][5/782] Loss_D: 2.7082 Loss_G: 7.0745
[0/25][6/782] Loss_D: 0.4328 Loss_G: 2.5920
[0/25][7/782] Loss_D: 1.4336 Loss_G: 6.2953
[0/25][8/782] Loss_D: 0.6306 Loss_G: 4.5373
[0/25][9/782] Loss_D: 0.3995 Loss_G: 3.8564
[0/25][10/782] Loss_D: 0.3725 Loss_G: 5.0983
[0/25][11/782] Loss_D: 0.5648 Loss_G: 2.9095
[0/25][12/782] Loss_D: 1.1040 Loss_G: 8.0325
[0/25][13/782] Loss_D: 0.7484 Loss_G: 6.1895
[0/25][14/782] Loss_D: 0.1610 Loss_G: 3.5891
[0/25][15/782] Loss_D: 0.5891 Loss_G: 6.1586
[0/25][16/782] Loss_D: 0.4395 Loss_G: 4.4962
[0/25][17/782] Loss_D: 0.3508 Loss_G: 4.4119
[0/25][18/782] Loss_D: 0.4131 Loss_G: 6.5251
[0/25][19/782] Loss_D: 0.3560 Loss_G: 4.1484
[0/25][20/782] Loss_D: 0.3766 Loss_G: 6.2311
[0/25][21/782] Loss_D: 0.1742 Loss_G: 5.6354
[0/25][22/782] Loss